# Query lexer. Very beta

It works rather well, almost surprisingly. Only with and or or, and parenthesis, I did not really test the quotes and not operator

In [14]:
"""Search query parser

version 2006-03-09

This search query parser uses the excellent Pyparsing module 
(http://pyparsing.sourceforge.net/) to parse search queries by users.
-------------------------------------------------------------------------------
Copyright (c) 2006, Estrate, the Netherlands
All rights reserved.

Redistribution and use in source and binary forms, with or without modification,
are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.
* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation 
  and/or other materials provided with the distribution.
* Neither the name of Estrate nor the names of its contributors may be used
  to endorse or promote products derived from this software without specific
  prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR
ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; 
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON 
ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT 
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS 
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

CONTRIBUTORS:
- Steven Mooij
- Rudolph Froger
- Paul McGuire


~~ End of notice

I did not alter the structure of this thing a lot. Mostly modified it to work in my implementation of the index,
which uses tuples docid + score instead of single ids. Thus some things might not work yet.
Also hurray for global variables and definitions, because I didn't want to write more classes.
"""
from pyparsing import Word, alphanums, Keyword, Group, Combine, Forward, Suppress, Optional, OneOrMore, oneOf

class SearchQueryParser:

    def __init__(self):
        self._methods = {
            'and': self.evaluateAnd,
            'or': self.evaluateOr,
            'not': self.evaluateNot,
            'parenthesis': self.evaluateParenthesis,
            'quotes': self.evaluateQuotes,
            'word': self.evaluateWord,
            'wordwildcard': self.evaluateWordWildcard,
        }
        self._parser = self.parser()
    
    def parser(self):
        operatorOr = Forward()
        
        operatorWord = Group(Combine(Word(alphanums) + Suppress('*'))).setResultsName('wordwildcard') | \
                            Group(Word(alphanums)).setResultsName('word')
        
        operatorQuotesContent = Forward()
        operatorQuotesContent << (
            (operatorWord + operatorQuotesContent) | operatorWord
        )
        
        operatorQuotes = Group(
            Suppress('"') + operatorQuotesContent + Suppress('"')
        ).setResultsName("quotes") | operatorWord
        
        operatorParenthesis = Group(
            (Suppress("(") + operatorOr + Suppress(")"))
        ).setResultsName("parenthesis") | operatorQuotes

        operatorNot = Forward()
        operatorNot << (Group(
            Suppress(Keyword("not", caseless=True)) + operatorNot
        ).setResultsName("not") | operatorParenthesis)

        operatorAnd = Forward()
        operatorAnd << (Group(
            operatorNot + Suppress(Keyword("and", caseless=True)) + operatorAnd
        ).setResultsName("and") | Group(
            operatorNot + OneOrMore(~oneOf("and or") + operatorAnd)
        ).setResultsName("and") | operatorNot)
        
        operatorOr << (Group(
            operatorAnd + Suppress(Keyword("or", caseless=True)) + operatorOr
        ).setResultsName("or") | operatorAnd)

        return operatorOr.parseString

    def evaluateAnd(self, argument):
        left = self.evaluate(argument[0])
        right = self.evaluate(argument[1])
        
        found = set(left.keys()).intersection(set(right.keys()))
        scores = {}
        for key in found:
            scores[key] = left[key] + right[key]
        return scores

    def evaluateOr(self, argument):
        scores = {}
        for i in range(0,2):
            localresults = self.evaluate(argument[i])
            for key in localresults.keys():
                    scores[key] = scores.setdefault(key, 0) + localresults[key]
        return scores

    def evaluateNot(self, argument):
        return self.GetNot(self.evaluate(argument[0]))

    def evaluateParenthesis(self, argument):
        return self.evaluate(argument[0])

    def evaluateQuotes(self, argument):
        r = Set()
        search_terms = []
        for item in argument:
            search_terms.append(item[0])
            if len(r) == 0:
                r = self.evaluate(item)
            else:
                r = r.intersection(self.evaluate(item))
        return self.GetQuotes(' '.join(search_terms), r)

    def evaluateWord(self, argument):
        return self.GetWord(argument[0])

    def evaluateWordWildcard(self, argument):
        return self.GetWordWildcard(argument[0])
        
    def evaluate(self, argument):
        return self._methods[argument.getName()](argument)

    def Parse(self, query):
        #print self._parser(query)[0]
        return self.evaluate(self._parser(query)[0])

    def GetWord(self, word):
        return query(word)

    def GetWordWildcard(self, word):
        return query(word)

    def GetQuotes(self, search_string, tmp_result):
        return query(search_string, mode="and")

    def GetNot(self, not_set):
        return Set().difference(not_set)

# Indexer and querying system

This notebook provides a fonctionnal system to index a bunch of text document, and to run fast text queries against the index.

Author : Anthony Rossi


## Imports

In [15]:
from nltk.corpus import stopwords
from nltk.stem import *
from os import listdir, mkdir
from os.path import isfile, join
from nltk.corpus import stopwords
from nltk.tokenize import *
from tqdm import *
from sklearn import feature_extraction    
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import operator
import pickle
import json
import sys
import mmap

## Definitions and global variables

In [16]:
stemmer = PorterStemmer()
cachedStopWords = stopwords.words("english")
docs = listdir("latimes_cleaned")
index = {}
indexed = {}
termcounts = {}
titles = {}
artcount = 0

In [17]:
def save_obj(obj, name ):
    with open('data_cache/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('data_cache/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [18]:
def mergeindex(globalindex, localindex, docid):
    for key in localindex.keys():
        globalindex.setdefault(key,[]).append((docid, localindex[key]))

def updatecount(globalcounts, localcounts):
    for key in localcounts.keys():
        globalcounts[key] = globalcounts.setdefault(key, 0) + 1

def parsestring(words):
    local_index = {}
    for word in words:
        if not word in cachedStopWords:
            stemmed = stemmer.stem(word)
            local_index[stemmed] = local_index.setdefault(stemmed, 0)+1
    return local_index


def parsefile(document):
    global artcount
    local_index = {}
    j = json.load(open(document))
    for art in j:
        artcount += 1
        words = []
        if "headline" in art.keys():
            words = words + word_tokenize(art["headline"].lower()) * 3
            titles[art["docno"]] = art["headline"]
        else : 
            titles[art["docno"]] = art["docno"]
        if "text" in art.keys():
            words = words + word_tokenize(art["text"].lower())
        lindex = parsestring(words)
        mergeindex(index,lindex, art["docno"])
        updatecount(termcounts, lindex)
        
        
def addToIndex(doc, index, indexed, termcount):
    if not doc in indexed.keys() :
        local_index = parsefile("latimes_cleaned/"+doc)
        indexed[doc] = True

## Interaction with the indexer

The block bellow will erase all the index and rebuild it from the files in the dataset. 
On my computer, it takes about 15 minutes to go through all the documents, so have a coffee machine handy if you plan to run this on a phone (it takes about 3 hours).

I totally recommend you to use the loading system instead, see bellow.

If, for some reason, you need to modify the index, here are a few tips :
* If you just need to add an entry, you can do so without rebuilding the whole thing. just load the index and use addToIndex
* If you are testing, use python's slice system to load only a few documents : replace docs with docs[:30] to load only the first 30 documents
* If you need to build the whole index, grab a coffee.

For now the whole thing sits in the RAM, just because. Obviously, I will have to change that.

In [36]:
# This bloc recreate the whole index. Use load instead if possible
def flushIndexToFile(fname):
    with open("partialPLS/"+str(fname),'w+b') as f:
        for key in sorted(index.keys()):
            j = json.dumps(index[key])
            toWrite = key+"\n"+j+"\n"
            f.write(toWrite.encode())
        f.flush()
        
def mergePartialPLs(count):
    global ifindex
    ifindex = {}
    offset = 0
    try:
        mfid.close()
        mfscore.close()
    except:
        pass # Python !
#TODO : mutex the hell out of that stuff !
    with open('PLByDocID', 'w+b') as f:
        with open('PLByScore', 'w+b') as g:
            cursors = []
            for fname in range(0, count):
                h =open("partialPLs/"+str(fname),'r+b')
                cursors.append([h.readline().decode().rstrip(), h])
            while len(cursors) > 0:
                cursors = sorted(cursors, key= operator.itemgetter(0))
                tempIndex = []
                tempRef = 0
                tempWord = cursors[0][0]
                while tempRef < len(cursors) and cursors[tempRef][0] == tempWord :
                    toParse = cursors[tempRef][1].readline().decode().rstrip()
                    try:
                        lind = json.loads(toParse)
                        for el in lind:
                            tempIndex.append(el)
                    except:
                        print("Error raised for word "+str(tempWord)+" in file "+str(tempRef))
                    
                    cursors[tempRef][0] = cursors[tempRef][1].readline().rstrip()
                    if cursors[tempRef][0] == b'':
                        cursors[tempRef][1].close()
                        del cursors[tempRef]
                        print("element removed")
                        tempRef -= 1 # Else we would skip a file
                    tempRef += 1
                j1 = json.dumps(sorted(tempIndex, key = operator.itemgetter(0))).encode()
                j2 = json.dumps(sorted(tempIndex, key = operator.itemgetter(1))[::-1]).encode()
                f.write(j1)
                g.write(j2)
                ifindex[tempWord] = (offset, len(j1))
                offset += len(j1)
            g.flush()
        f.flush()

docs = listdir("latimes_cleaned/")
index = {}
indexed = {}
termcounts = {}
titles = {}
artcount = 0
trig = False
partialPL = 0
try:
    mkdir("partialPLs")
except:
    pass
for doc in tqdm(docs[:10]) : # I totally love tqdm.
    addToIndex(doc, index, indexed, termcounts)
    
    if sys.getsizeof(index) > 300000: #Comment out to restore old behavior
        flushIndexToFile(partialPL)
        partialPL += 1
        index = {}
if len(index)>0:        
    flushIndexToFile(partialPL)
    partialPL +=1
mergePartialPLs(partialPL)        



100%|██████████████████████████████████████████| 10/10 [00:13<00:00,  1.52s/it]


element removed
element removed
element removed
element removed
element removed
element removed
element removed
element removed
element removed
element removed


The block bellow saves the whole index as a pickle file.
This way, you can load the index afterwards, and save a precious time ! (see what I did here ? Cause building the index takes time, and it's a file that you're saving, so saving time)

In [37]:
def save():
    all_index = (index, indexed, termcounts, titles, artcount, ifindex)
    save_obj(all_index, "indexFile_cleaneddata")
save()

And the block bellow loads all the index from disk. about 3 seconds for the whole thing on my computer

In [23]:
all_index = load_obj("indexFile_cleaneddata")
index, indexed, termcounts, titles, artcount, ifindex = all_index
print("successfully imported {count} indexed documents".format(count=len(indexed)))

successfully imported 10 indexed documents


## Query system

Bellow is the querying system. Use it to, well, query the index

In [38]:
with open('PLByDocID', 'r+b') as f:
    mfid = mmap.mmap(f.fileno(),0)
with open('PLByScore', 'r+b') as g:
    mfscore = mmap.mmap(g.fileno(),0)

def getWordPL(word):
    try:
        offset, length = ifindex[word]
        mfid.seek(offset)
        j = mfid.read(length)
        rbind = json.loads(j.decode())
        return rbind
    except:
        return None

def query(request, mode="or"):
    words = request.lower().split()
    scores = {}
    matches = []
    global artcount
    for word in tqdm(words):
        stemmed = stemmer.stem(word)
        
        ids = []
        if stemmed in termcounts.keys():
            localresults = getWordPL(stemmed)
            idf = math.log(artcount/termcounts[stemmed])

            for match in localresults:
                docid, score = match
                ids.append(docid)
                score = score * idf
                scores[docid] = scores.setdefault(docid, 0) + score
        matches.append(ids)
    if mode == "and":
        found = set(matches[0]).intersection(*matches)
        notfound = list(set(scores)-set(found))
        for miss in notfound:
            del scores[miss]
    return scores

def getBestSimple(request, nbresults=10, mode="or"):
    scores = query(request, mode)
    return sorted(scores.items(), key=operator.itemgetter(1))[:-nbresults:-1]

def translateTitles(response):
    modified = []
    for pair in response:
        docno, score = pair
        modified.append((titles[docno], score))
    return modified

### Simple queries

This is where you enter the queries for now. This system supports only simple queries, composed by a list of words and a mode of selection (or or and).

There is a much better query system bellow. with a lexer, a parser, parenthesis, and a few other things.

In [39]:
getBestSimple("towel", mode="and")

  0%|                                                    | 0/1 [00:00<?, ?it/s]


TypeError: 'NoneType' object is not iterable

### Complex queries

This other system is more advanced, and features a lexer and a parser to support complex queries with various operator and parenthesis.

In [26]:

parserTest = SearchQueryParser()
def getBest(request, nbresults=10):
    scores = parserTest.Parse(request)
    return sorted(scores.items(), key=operator.itemgetter(1))[:-nbresults:-1]

In [28]:
translateTitles(getBest('towel', 10))

  0%|                                                    | 0/1 [00:00<?, ?it/s]


TypeError: 'NoneType' object is not iterable

In [29]:
getBest('towel', 10)

  0%|                                                    | 0/1 [00:00<?, ?it/s]


TypeError: 'NoneType' object is not iterable

## Clustering

### Calculate distance

Transform documents listed in the query results into tf-idf vectors, then compute the cosine similarity between them 

In [34]:
def getResultDocs(results):
    rDocuments = []
    for result in results :
        docno, score = result
        rDocuments.append(docno)
    return rDocuments

def getTfidf (documents):
    tf  = []
    idf = []
    

In [35]:
getTfidf(getResultDocs(getBest('towel',10)))

  0%|                                                    | 0/1 [00:00<?, ?it/s]


TypeError: 'NoneType' object is not iterable

In [40]:
ifindex

{b'foreign-born': (2620473, 44),
 b'vaclav': (6201234, 132),
 b'canvas': (1393863, 44),
 b'3,500': (434828, 198),
 b':22': (603053, 22),
 b'dump': (2169007, 639),
 b'facilit': (2422858, 220),
 b'work-forc': (6456484, 22),
 b'89-87': (572093, 44),
 b'suprem': (5736948, 616),
 b'fickl': (2500629, 22),
 b'double/queen': (2129254, 22),
 b'gotch': (2827488, 22),
 b'offshor': (4219612, 110),
 b'digress': (2059799, 22),
 b'courtrier': (1825170, 22),
 b'bakar': (1068686, 22),
 b'relent': (4919165, 44),
 b'two-hous': (6104898, 22),
 b'wirth-heinz': (6418539, 22),
 b'74-72': (551831, 22),
 b'wanton': (6291165, 22),
 b'auschwitz': (1018668, 45),
 b'tenet': (5843831, 22),
 b'quick-think': (4773562, 22),
 b'8.03': (562897, 44),
 b'19.7': (324155, 22),
 b'uncount': (6129537, 22),
 b'157': (302859, 22),
 b'achil': (671539, 22),
 b'beekeep': (1131822, 22),
 b'boni': (1239053, 44),
 b'jog': (3300347, 198),
 b'husar': (3092267, 22),
 b'joyner-kerse': (3318634, 22),
 b'albania': (770850, 66),
 b'keeler':